In [1]:
import os.path
import sys
import django
sys.path.append('/home/galm/software/django/tmv/BasicBrowser')
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
import scoping
from scoping.models import *

import pandas as pd

In [2]:
# Define the query ID we re looking at
q = Query.objects.get(pk=10452)
p = q.project

# Get the document IDs attached to this query
dids = set(Doc.objects.filter(query=q).values_list('pk', flat=True))
print(len(dids))

49716


In [3]:
import statistics

django.db.connection.close()

def mode_na(x):
    try:
        return statistics.mode([y for y in x if y is not np.NaN])
    except:
        return np.NaN
    
columns = [
    'id',
    'content',
    'title',
    'wosarticle__de',
    'wosarticle__wc'
]


# Get all of the document ratings in this query
dos = DocOwnership.objects.filter(
    query=q, 
    relevant__gt=0,
    doc__content__iregex='\w'
)
dodf = pd.DataFrame.from_dict(dos.order_by('finish').values(
    'id','doc_id','user__username','relevant'
))
dodf.loc[dodf['relevant']==2,'relevant'] = 0
dodf.loc[dodf['relevant']==3,'relevant'] = np.NaN

# Aggregate by document, and set the value of relevant to the most common selection, or NA if there is no mode
seen_df = dodf.groupby('doc_id').agg(lambda x: mode_na(x)).reset_index()[['doc_id','relevant']].rename(columns={"doc_id":"id"})
seen_df['seen'] = 1
seen_df.loc[pd.isna(seen_df['relevant']),'seen'] = np.NaN

# Get the document info
ddf = pd.DataFrame.from_dict(
    Doc.objects.filter(pk__in=seen_df['id']).values(*columns)
)
# And merge this with the majority relevance decision
seen_df = pd.merge(ddf,seen_df)

# Save to a csv
seen_df.to_csv('data/0_labelled_documents.csv',index=False)
print(seen_df.shape)
seen_df.head()

(11764, 7)


,id,content,title,wosarticle__de,wosarticle__wc,relevant,seen
0,7,In order to evaluate the effects of pH reducti...,Simulating ocean acidification and CO2 leakage...,CO2-driven acidification; Rank species sensiti...,[Environmental Sciences],0.0,1.0
1,385,Ants are being widely studied in different Bra...,Effect of climatic variables and vine crops on...,Abiotic factors; Agroecosystems; Bioindicators...,None,0.0,1.0
2,945,Forest transition is the change from net defor...,Ecological and social consequences of the Fore...,Deforestation; Development; Indigenous people;...,[Environmental Studies],0.0,1.0
3,1124,Diversification of the farm income base repres...,Spatial differentiation of farm diversificatio...,Diversification; Farm abandonment; Farm activi...,[Environmental Studies],0.0,1.0
4,1247,"As a main rural initiative, village land share...",Rural development led by autonomous village la...,built environment; China; high-density regions...,None,0.0,1.0


In [4]:
import matplotlib.pyplot as plt
#show how many relevant and irrelevant documents we have
seen_df.groupby('relevant').size().plot.bar()

In [5]:
# Get all the documents that have not yet been seen and save these as another csv
unseen_docs = Doc.objects.filter(
    id__in=dids,
    content__iregex='\w',
).exclude(pk__in=seen_df['id'])

unseen_df = pd.DataFrame.from_dict(list(unseen_docs.values(*columns)))
unseen_df = unseen_df[columns]
unseen_df['relevant'] = 0
unseen_df['seen'] = 0
unseen_df['random_sample'] = 0

print(unseen_df.shape)
unseen_df.to_csv('data/0_unlabelled_documents.csv', index=False)
unseen_df.head()

(37679, 8)


,id,content,title,wosarticle__de,wosarticle__wc,relevant,seen,random_sample
0,179,1. Biodiversity conservation of grasslands in ...,Influence of land use on plant community compo...,communal land; livestock grazing; plantation f...,[Biodiversity Conservation; Ecology],0,0,0
1,1378,"Robinia pseudoacacia, invaded many countries a...",Black locust (Robinia pseudoacacia) beloved an...,Robinia pseudoacacia; Central Europe; History;...,[Forestry],0,0,0
2,1958,The current and future security landscape is d...,Introduction,artifact; biography; Ottoman empire; theoretic...,None,0,0,0
3,2144,While both academics and politicians have long...,Feeding unrest: Disentangling the causal relat...,Africa; food prices; food security; instrument...,[International Relations; Political Science],0,0,0
4,2243,Veerasamy has made several comments concerning...,Reply,adrenalin; food allergen; allergic reaction; a...,None,0,0,0
